In [587]:
import falcon
import ntt

from common import q
from numpy import set_printoptions
from math import sqrt
from fft import fft, ifft, sub, neg, add_fft, mul_fft
from ntt import sub_zq, mul_zq, div_zq
from ffsampling import gram, ffldl_fft, ffsampling_fft
from ntrugen import ntru_gen
from encoding import compress, decompress
# https://pycryptodome.readthedocs.io/en/latest/src/hash/shake256.html
from Crypto.Hash import SHAKE256, SHA512
# Randomness
from os import urandom
from rng import ChaCha20
# For debugging purposes
import sys
if sys.version_info >= (3, 4):
    from importlib import reload  # Python 3.4+ only.

set_printoptions(linewidth=200, precision=5, suppress=True)

logn = {
    2: 1,
    4: 2,
    8: 3,
    16: 4,
    32: 5,
    64: 6,
    128: 7,
    256: 8,
    512: 9,
    1024: 10
}

# Bytelength of the signing salt and header
HEAD_LEN = 1
SALT_LEN = 40
SEED_LEN = 56

# Parameter sets for Falcon:
# - n is the dimension/degree of the cyclotomic ring
# - sigma is the std. dev. of signatures (Gaussians over a lattice)
# - sigmin is a lower bounds on the std. dev. of each Gaussian over Z
# - sigbound is the upper bound on ||s0||^2 + ||s1||^2
# - sig_bytelen is the bytelength of signatures
Params = {
    # FalconParam(2, 2)
    2: {
        "n": 2,
        "sigma": 144.81253976308423,
        "sigmin": 1.1165085072329104,
        "sig_bound": 101498,
        "sig_bytelen": 44,
    },
    # FalconParam(4, 2)
    4: {
        "n": 4,
        "sigma": 146.83798833523608,
        "sigmin": 1.1321247692325274,
        "sig_bound": 208714,
        "sig_bytelen": 47,
    },
    # FalconParam(8, 2)
    8: {
        "n": 8,
        "sigma": 148.83587593064718,
        "sigmin": 1.147528535373367,
        "sig_bound": 428865,
        "sig_bytelen": 52,
    },
    # FalconParam(16, 4)
    16: {
        "n": 16,
        "sigma": 151.78340713845503,
        "sigmin": 1.170254078853483,
        "sig_bound": 892039,
        "sig_bytelen": 63,
    },
    # FalconParam(32, 8)
    32: {
        "n": 32,
        "sigma": 154.6747794602761,
        "sigmin": 1.1925466358390344,
        "sig_bound": 1852696,
        "sig_bytelen": 82,
    },
    # FalconParam(64, 16)
    64: {
        "n": 64,
        "sigma": 157.51308555044122,
        "sigmin": 1.2144300507766141,
        "sig_bound": 3842630,
        "sig_bytelen": 122,
    },
    # FalconParam(128, 32)
    128: {
        "n": 128,
        "sigma": 160.30114421975344,
        "sigmin": 1.235926056771981,
        "sig_bound": 7959734,
        "sig_bytelen": 200,
    },
    # FalconParam(256, 64)
    256: {
        "n": 256,
        "sigma": 163.04153322607107,
        "sigmin": 1.2570545284063217,
        "sig_bound": 16468416,
        "sig_bytelen": 356,
    },
    # FalconParam(512, 128)
    512: {
        "n": 512,
        "sigma": 165.7366171829776,
        "sigmin": 1.2778336969128337,
        "sig_bound": 34034726,
        "sig_bytelen": 666,
    },
    # FalconParam(1024, 256)
    1024: {
        "n": 1024,
        "sigma": 168.38857144654395,
        "sigmin": 1.298280334344292,
        "sig_bound": 70265242,
        "sig_bytelen": 1280,
    },
}

In [588]:
import random

def seeded_rng(i):
    random_generator = random.Random(42)
    return bytes(random_generator.getrandbits(8) for _ in range(i))

def zero_rng(i):
    return b'\x00'*i

In [589]:
n = 512
sk = falcon.SecretKey(n)
pk = falcon.PublicKey(sk)

In [590]:
byte_string = b'test'
signature = sk.sign(byte_string, seeded_rng)
sk.verify(byte_string, signature)

True

In [591]:
def sign_pk_recovery(sk, message, randombytes=urandom):
    """
    Sign a message. The message MUST be a byte string or byte array.
    Optionally, one can select the source of (pseudo-)randomness used
    (default: urandom).
    """
    int_header = 0x30 + logn[sk.n]
    header = int_header.to_bytes(1, "little")

    salt = randombytes(SALT_LEN)
    hashed = sk.hash_to_point(message, salt)

    # We repeat the signing procedure until we find a signature that is
    # short enough (both the Euclidean norm and the bytelength)
    while(1):
        if (randombytes == urandom):
            s = sk.sample_preimage(hashed)
        else:
            seed = randombytes(SEED_LEN)
            s = sk.sample_preimage(hashed, seed=seed)
        norm_sign = sum(coef ** 2 for coef in s[0])
        norm_sign += sum(coef ** 2 for coef in s[1])
        # Check the Euclidean norm
        if norm_sign <= sk.signature_bound:
            # Check that s2 (s[1]) is invertible
            ntt.intt(ntt.ntt(s[1]))
            # try:
            #     ntt.intt(ntt.ntt(s[1]))
            # except: # unsure what error it would cause if it's not invertible
            #     continue
            
            # signature is (compress(s1), compress(s2), r)
            enc_s1 = compress(s[0], sk.sig_bytelen - HEAD_LEN - SALT_LEN)
            enc_s2 = compress(s[1], sk.sig_bytelen - HEAD_LEN - SALT_LEN)
            
            
            # Check that the encoding is valid (sometimes it fails)
            if enc_s1 and enc_s2:
                return header + enc_s1 + enc_s2 + salt
            
            
            # enc_s = compress(s[1], sk.sig_bytelen - HEAD_LEN - SALT_LEN)
            # # Check that the encoding is valid (sometimes it fails)
            # if (enc_s is not False):
            #     return header + salt + enc_s

In [592]:
def verify_pk_recovery(sk, pk, message, signature):
    """
    Verify a signature.
    """
    # # Unpack the salt and the short polynomial s1
    # salt = signature[HEAD_LEN:HEAD_LEN + SALT_LEN]
    # enc_s = signature[HEAD_LEN + SALT_LEN:]
    # s1 = decompress(enc_s, sk.sig_bytelen - HEAD_LEN - SALT_LEN, sk.n)
    signature_length = sk.sig_bytelen - HEAD_LEN - SALT_LEN
    
    enc_s1 = signature[HEAD_LEN: signature_length + HEAD_LEN]
    enc_s2 = signature[signature_length + HEAD_LEN: 2*signature_length + HEAD_LEN]
    salt = signature[2*signature_length + HEAD_LEN:]
    
    # Need to unpack polynomial s1 and s2
    s1 = decompress(enc_s1, signature_length, sk.n)
    s2 = decompress(enc_s2, signature_length, sk.n)
    
    ## Check that s1 and s2 are valid
    if not s1 or not s2:
        print("Invalid encoding")
        return False

    # Check that the (s1, s2) is short
    norm_sign = sum(coef ** 2 for coef in s1)
    norm_sign += sum(coef ** 2 for coef in s2)
    if norm_sign > sk.signature_bound:
        print("Squared norm of signature is too large:", norm_sign)
        return False
    
    # Check that pk = H(inverse(s2)*(HashToPoint(r||m, q, n) - s1))
    
    hash_to_point_message = sk.hash_to_point(message, salt)
    recovered_pk = div_zq(sub_zq(hash_to_point_message, s1),s2)
    
    return recovered_pk == pk.h

The encoded values are concatenated into a bit sequence of 14n bits, which is then represented as ⌈14n/8⌉ bytes.

In [593]:
signature_pkr = sign_pk_recovery(sk, byte_string, seeded_rng)

In [594]:
verify_pk_recovery(sk, pk, byte_string, signature_pkr)

True

Additional Tests

In [595]:
f = [
-4, -2, -5, -1, 4, -2, 0, -3, -1, 1, -2, -2, -6, -3, 3, -5, -1, 4, -3, -8, 4, -1, 2,
-1, -8, 5, -6, -3, 6, 0, -2, 4, 5, -6, 2, 3, 6, 4, 2, 3, 3, 7, 0, 1, 5, -3, -1, -9, -1,
6, -2, -5, 4, 0, 4, -2, 10, -4, -3, 4, -7, -1, -7, -2, -1, -6, 5, -1, -9, 3, 2, -5, 4,
-2, 2, -4, 4, -3, -1, 0, 5, 2, 2, -1, -9, -7, -2, -1, 0, 3, 1, 0, -1, -2, -5, 4, -1,
-1, 3, -1, 1, 4, -3, 2, -5, -2, 2, -4, 3, 6, 3, 9, 1, -2, 4, -1, -1, -6, -2, -2, 4, 5,
-1, 0, 10, -2, 1, -2, -3, 0, -4, -4, -1, 0, 1, -5, -3, -7, -2, -1, 2, -6, 3, 0, 0, 4,
-4, 0, 0, -5, -2, 5, -8, 8, 5, 4, 10, -4, 3, 8, 5, 1, -7, 0, -5, 0, -4, 3, -4, -2, 2,
-2, 6, 8, 2, -1, 4, -4, -2, 1, 0, 3, 7, 0, 9, -3, 1, 4, -3, 2, -1, 5, -8, 4, -1, 1, -8,
2, 4, -9, -3, 1, 3, -1, -7, 5, 5, 4, -3, 0, -7, -3, -1, -6, -7, 0, -3, 0, 3, -3, 0, -3,
1, 3, 4, -6, -6, -3, 6, 0, 2, -5, 1, -3, -6, -6, -1, -7, -2, -4, 3, 0, -4, -1, 2, 7,
-7, -2, 4, 2, 0, 1, -1, -3, 2, 1, 8, -1, 1, -2, 1, -1, 1, 4, 0, -4, 4, 3, -2, 6, -3,
-2, 1, 2, 3, 6, 5, -4, -7, -6, 4, 3, -4, 3, -3, 3, -3, 2, -1, 1, 5, -2, 2, 1, 0, -7, 0,
0, -1, 4, -3, 2, 1, -3, 5, 4, -6, -1, -3, 2, -1, -8, 4, 2, 4, 0, 1, -5, 8, 5, 4, -3,
-1, -2, 4, 0, 2, -2, 0, -2, -1, -7, 5, 0, 1, 2, 1, -2, 2, -1, 1, -4, 1, 0, 4, -4, 0, 5,
1, 4, -5, -2, -3, -2, 1, 3, 1, 2, 5, 12, 0, -1, 4, -6, 1, -4, 3, -5, -4, 4, 2, -2, -6,
1, 1, 3, -1, 0, -4, -4, -4, 6, -2, 4, -3, 0, -2, -1, 0, -6, -3, -2, 0, 6, 5, -5, -5, 3,
0, 3, -3, -2, 5, 7, -3, 1, -1, 0, 3, 0, 3, -7, 2, -4, -4, 1, 1, 1, 0, -3, -8, 3, 6, 1,
-2, -7, 3, 3, 4, -1, -2, -5, 9, 7, 1, 2, -4, 4, 0, -11, 3, 0, -3, -5, 5, -1, -1, 7, 6,
-1, 6, 3, 9, 5, -2, -3, -3, 1, -2, 0, -1, 1, -2, 2, 0, -5, -1, -4, -2, 2, -1, -3, 0,
-3, 0, 1, 3, -3, 2, 5, 8, -2, 3, -4, -7, 0, 4, -8, 1, 8, -2, 1, -1, 2, 0, -2, 1, 3, 3,
4, -2, -4, 3, -4, 2, 3, -2, -4, 1, -4, 10, 2,
]
g =[-1, 5, -7, -1, -4, 6, 4, -1, -4, -13, -1, -5, -2, -8, 2, 1, 4, 2, 0, 0, 2, 0, -1, 2, 5, -5, -8, 8, 1, 11, 0, -8, -4, 1, 1, -6, -4, 1, -3, 0, -10, -4, -6, -3, -2, 1, 6, 2, 8, -2, 2, -2, 1, 3, -4, 2, -1, -1, -2, -2, -3, 0, -3, 2, -3, 2, -3, -4, 2, 3, 4, -5, 6, -3, -2, -1, -1, -6, -2, 1, -4, -7, 8, 0, 2, -2, 2, 0, 1, 0, 4, 9, 7, 0, -1, -1, 4, -3, -2, 6, 6, 0, 1, 7, -6, -5, 5, 1, 4, -1, 0, -2, 3, -4, 1, -1, -3, -2, 0, -1, -7, -8, -1, 2, 0, -5, 0, 1, -4, 6, -5, 6, 4, 1, -4, -5, 8, -1, 1, -2, 1, 1, 1, 3, 0, -1, 1, 1, -4,
-5, -4, 2, -3, 2, -2, 3, 7, -4, 4, -1, -2, 4, -4, -5, 2, 6, -7, 5, -1, 1, 3, 0, -5, -5,
3, -2, -3, -1, -6, 0, 2, 3, 2, 7, -3, -2, -2, 1, -5, 3, 3, -7, 0, 4, 4, -1, 2, -3, 1,
3, -1, -1, 0, -7, -6, -3, 7, -3, 5, -5, 1, -2, 0, 9, -2, 3, -1, -5, -3, -5, 3, 1, -4,
-3, 2, -2, 2, 8, -1, 0, 5, -3, -2, -6, 4, 0, 3, -3, -3, 4, -1, 0, 0, -2, -1, 3, 7, 4,
5, -1, 8, 0, -1, -6, -3, 4, 3, -3, 5, 2, -1, -2, 1, -1, 3, -2, -6, 4, 0, 0, -4, 1, 6,
2, 0, 10, 9, 2, -2, 0, 2, 1, -3, -1, -1, 3, 2, 1, 1, -3, -2, 7, 2, -1, 5, -3, -2, 1,
-2, 2, -2, -4, 3, 2, 1, -4, 1, 4, 3, -7, -4, 2, -5, -2, 5, -3, 1, -4, -5, 1, 0, 0, 0,
7, -5, -1, 2, 2, -3, 6, -6, 4, -3, -5, -6, -7, -4, 3, -2, -2, -10, -3, 2, -1, -6, -4,
1, 2, 2, 1, 4, 1, -5, -10, -2, 2, -4, 4, 4, -2, 1, 4, -3, 0, -6, -3, 1, 5, -7, -6, -4,
8, -1, 0, -1, 6, -3, -2, -2, 6, 2, 3, -3, -3, 5, -2, 1, 1, -4, -4, 8, 0, 3, 2, 3, 7, 4,
3, 2, -6, -9, 0, -8, 11, -2, 2, -2, -2, 3, 0, -6, 2, -1, 4, 2, -2, 0, -3, -7, -1, -1,
0, -1, -4, -2, -5, 3, -4, 2, 2, -1, -1, 7, -1, 3, 6, -7, 1, -5, 0, -7, 4, 3, -5, -1, 0,
3, -4, 1, 2, -7, 1, -2, -8, -2, -5, -5, 1, -4, -4, 4, -3, -2, 2, -4, -8, -1, 0, -9, 5,
-1, -2, 3, 2, 6, -1, 1, -1, -5, 5, 9, 3, -6, -5, 1, -6, 0, 2, -4, 6, 2, 7, 2, 15, 0,
-2, 9, 0, 1, 6, 4, -1, -1, -6, -3, 3, 1, -6, -3, 2, 2, -2,
];
capital_f = [
0, -25, -39, 21, 7, -5, -10, 4, -1, -38, -9, -1, 4, -23, 15, -1, 8, 1, -38, 41, 29, 22,
9, 12, -46, 0, 9, -17, -19, 32, 38, -3, 14, 6, 2, -6, -18, -1, 23, 80, -12, -20, 24,
22, -31, -38, -11, 8, 17, 18, 19, -10, 0, -1, 28, -5, -28, -33, 4, -31, -33, -8, -9,
-44, 46, -11, -5, -21, -22, -7, 1, -11, 33, -8, 12, -7, -6, 63, 17, 12, -49, -11, -31,
-8, 7, -28, 33, -28, -19, 8, 46, -73, 9, 32, 18, 7, -43, 0, -6, -4, 8, -39, -17, 11,
15, -25, -9, -28, -2, 24, -23, 10, -15, 4, 41, 46, 18, 2, -3, -29, 11, -3, 20, 35, 21,
23, 5, -8, -3, -27, -69, 0, 26, -29, -24, 8, 19, 6, -14, -18, 47, 5, 21, -50, 17, -44,
-36, 24, 9, 16, -38, -5, -54, 34, 13, 31, -2, 9, 8, -12, -14, -17, 28, -59, -20, 19,
31, 14, 14, 7, -32, 37, 5, -3, -7, -6, 21, -29, -33, 23, -25, -23, 14, 38, -29, -33,
-9, 23, -43, 18, -12, 2, 30, 32, -28, -21, 42, 1, 6, -6, 58, 34, -22, 1, 5, -2, -8, 14,
-19, -4, -6, 10, -3, -3, 32, 18, -19, -12, 49, 13, 4, -18, 57, 37, -19, 25, 14, 18,
-51, 13, 4, 4, 17, -37, -2, 1, 41, -36, -8, -13, 49, -6, 9, 46, -36, -6, -20, -18, -6,
-29, -42, -21, -25, -29, 5, -41, 51, 49, -20, -22, -9, 3, -6, -52, 10, 41, 12, -27,
-20, 31, -17, -23, -16, 3, 44, -3, -5, -2, 0, -22, 14, -30, -41, 3, -27, 3, 18, 38, 10,
49, 45, -13, -27, -4, -10, -67, -1, -17, -2, 72, 46, 20, 24, 22, 16, 25, 6, -6, -31, 2,
0, -13, -14, 9, 4, 31, 18, 22, 12, 59, -1, -3, -24, -47, -10, 48, 37, -34, -32, -4, 18,
-2, 52, -8, -7, 34, -44, -14, -21, -49, -35, 41, -4, 31, 3, 23, 9, 8, 0, -24, 38, -9,
-9, 4, -10, -55, -19, 21, 27, 22, 41, 6, -23, 41, -2, 28, -46, 20, 52, 16, 20, 32, 18,
2, -3, 9, 16, 33, -18, 12, 6, -9, -19, 1, -5, -15, -17, 6, -3, 4, -22, 30, -34, 43, -4,
9, -3, -33, -43, -5, -13, -56, 38, 16, 11, -36, 11, -4, -56, 2, 0, -19, -45, -8, -34,
16, 31, -3, 16, 27, -16, -9, 8, 45, -51, -20, 62, -17, -4, 4, 17, -45, 4, -15, -19, 39,
39, 15, 17, -19, 2, 45, 36, -22, 16, -23, 28, 34, 12, 5, 10, -7, 28, -35, 17, -37, -50,
-28, 19, -25, 9, 45, -6, -7, -16, 57, 27, 50, -30, 2, -10, -1, -57, -49, -23, 0, -9,
-36, -4, -3, 32, -6, -25, 67, -27, -19, 25, -6, 1, -17, -14, 0, 29, 26, -12, -20, 44,
14, 10, 8, -11, -18, -53, 22, 25, 27, 35, 6, -16, 12, 71, -8,
]
capital_g = [
27, 6, 12, -3, -31, -42, 27, 17, 11, 8, 34, 6, -3, 2, 11, -11, 18, 48, 1, 21, -7, -6,
9, 33, -18, -40, -55, -9, -71, -50, 32, -36, 11, 4, 29, 33, 10, -19, -43, -10, 22, -36,
-23, -21, -14, -47, 25, -4, -14, 30, 16, -18, -11, 6, -37, -27, -12, 6, 7, 33, -36, 33,
-2, 12, -21, 1, 16, 49, -11, -16, -41, 15, 11, 8, 20, -15, 26, -8, 11, -43, -36, 28, 2,
-47, -30, -47, -1, 1, 48, -6, -22, 24, -20, -3, -1, -15, -12, 62, 12, 7, -9, 15, -71,
49, 22, 27, 20, -8, -28, -13, -31, 18, 28, 54, 29, 5, 0, 33, -5, -22, -21, -12, -14,
-2, 11, -24, 32, -26, -71, 21, -15, -20, -12, 36, -5, 35, 46, 13, -34, -8, 10, -10, 10,
40, -52, 8, 0, 18, -33, -10, 8, 43, -8, -6, -31, -17, 19, 30, 12, -9, 8, -19, -32, -18,
-1, -37, 4, 43, 27, 14, -6, -14, -44, -34, -8, 16, -39, 13, 6, -32, 8, 17, -12, 23,
-44, -25, -66, -12, -31, 30, 14, -9, -5, -10, 44, -12, -2, -43, -22, -18, -7, -9, -15,
-7, -21, -27, -5, 1, -13, -10, 8, -8, 29, 21, 64, 47, -28, -9, -28, 25, -47, -34, -3,
-14, -26, -12, -5, -10, -27, -9, -14, -23, -2, -31, 28, 17, -4, -30, 31, 3, -15, 25, 9,
-32, 0, -6, -22, 20, -37, 3, 12, -19, -17, 13, 30, 11, -15, 15, 50, 66, -31, -31, 16,
2, 3, -8, 40, -21, -31, -2, 41, -29, -12, 9, 14, -4, 9, 8, -20, 28, 12, 20, -10, 5, -6,
-33, 6, 21, 51, 30, 9, 3, 8, 7, 19, -53, 19, 15, 4, -38, 19, 29, 18, 6, 19, 3, -17,
-32, 16, 3, 46, -6, -3, 47, 3, -66, 3, 25, -6, -6, 21, -24, -9, 28, -39, -42, 42, -6,
-19, -14, 6, -8, 9, 28, -4, 23, 12, -17, -13, 3, 3, 6, 44, 6, -5, 38, -4, -16, 12, -15,
8, -11, 45, 1, -16, 37, -35, 20, 26, 9, 13, 34, 25, -3, -10, -2, -42, -23, -22, -56,
-56, 6, 17, -9, 0, 36, 20, 6, -58, 12, 0, -3, -29, -49, -24, -12, -13, 5, -39, -8, 36,
-9, 44, 35, -64, -22, -12, 26, -15, 41, 36, -19, -37, -20, 46, 35, 9, 32, -5, 27, 21,
-36, -51, 19, 10, -23, 28, 46, 28, 8, 22, -31, 18, 2, -16, -9, 1, -22, -22, 31, 14, 5,
44, -3, 38, 0, -12, 50, -23, -19, 1, 42, 15, 1, 13, 32, 45, 37, 15, 11, -9, -23, -6,
-23, 36, 4, -34, -14, -14, -37, -28, 19, 20, 14, 24, -48, -34, -27, -34, -12, 9, -20,
-30, 25, 28, -51, -13, 11, -20, -1, -3, 6, -38, -46, -15, 28, 10, -4, 3, -1, 4, -40,
16, 61, 31, 28, 8, -2, 21, -3, -25, -12, -32, -15, -38, 20, -7, -35, 28, 29, 9, -27,
]
n = 512
sk = falcon.SecretKey(n, [f, g, capital_f, capital_g])
pk = falcon.PublicKey(sk)

In [596]:
msg = b'\x00'*15

In [597]:
salt = b'\x00'*40

In [598]:
signature = sign_pk_recovery(sk, msg, zero_rng)

In [599]:
signature_length = 625

enc_s1 = signature[HEAD_LEN: signature_length + HEAD_LEN]
enc_s2 = signature[signature_length + HEAD_LEN: 2*signature_length + HEAD_LEN]
salt = signature[2*signature_length + HEAD_LEN:]

# Need to unpack polynomial s1 and s2
s1 = decompress(enc_s1, signature_length, sk.n)
s2 = decompress(enc_s2, signature_length, sk.n)

In [600]:
s_test_compress = compress(s2, 625)

In [601]:
s_test_decompress = decompress(s_test_compress, 625,512)

In [602]:
s2[:10]

[-101, 95, 2, 142, -3, 237, -135, -427, -223, -31]

In [603]:
s_test_compress[:10]

b'\xe5\xaf\xc0\xa1\xcc\x1d\xb5\x87j\xc7'

In [604]:
s_test_decompress[:10]

[-101, 95, 2, 142, -3, 237, -135, -427, -223, -31]

In [605]:
s1_ntt = ntt.ntt(s1)
print("s1")
print(s1[:10])
print(s1_ntt[:10])

s2_ntt = ntt.ntt(s2)
print("s2")
print(s2[:10])
print(s2_ntt[:10])

s1
[-186, -91, 274, 93, 71, 77, -100, -39, -56, -42]
[6934, 3452, 3265, 11623, 6876, 5489, 1690, 8186, 9761, 5972]
s2
[-101, 95, 2, 142, -3, 237, -135, -427, -223, -31]
[2188, 1684, 2413, 1481, 11617, 7847, 9617, 8184, 5112, 5620]


In [606]:
c = sk.hash_to_point(msg, salt)
recovered_pk = div_zq(sub_zq(c, s1),s2)

In [607]:
c[:10]

[1889, 5306, 8580, 2712, 2186, 6176, 11688, 5399, 2712, 11837]

In [608]:
recovered_pk[:10]

[179, 8726, 12176, 11472, 5106, 1218, 4892, 7443, 483, 11424]

In [609]:
c = sk.hash_to_point(msg, salt)
c_ntt = ntt.ntt(c)
test2 = ntt.sub_ntt(c_ntt, s1_ntt)
test3 = ntt.div_ntt(test2,s2_ntt)
test4 = ntt.intt(test3)

In [610]:
h_ntt = ntt.ntt(pk.h)

In [611]:
c_recalc = ntt.intt(ntt.add_ntt(s1_ntt,ntt.mul_ntt(s2_ntt,h_ntt)))

In [612]:
c_recalc[:10]

[1889, 5306, 8580, 2712, 2186, 6176, 11688, 5399, 2712, 11837]

In [613]:
print(c[:10])
print(c[:10])
print(test2[:10])
print(test3[:10])
print(test4[:10])

[1889, 5306, 8580, 2712, 2186, 6176, 11688, 5399, 2712, 11837]
[1889, 5306, 8580, 2712, 2186, 6176, 11688, 5399, 2712, 11837]
[10632, 2142, 9344, 10890, 2139, 7352, 1786, 10484, 2776, 10438]
[8879, 3723, 1547, 2812, 11024, 4873, 10789, 10134, 8674, 3190]
[179, 8726, 12176, 11472, 5106, 1218, 4892, 7443, 483, 11424]


In [614]:
s1= [153, 89, 54, 219, 74, -306, 152, 77, -183, 76, 232, 76, 341, 338, 243, -205, -474, -115, 222, 45, 138, -77, -96, 126, -87, -247, 149, 79, 290, -155, 115, 253, 426, 72, 116, 102, -23, -168, -6, 132, -25, 18, -118, 98, 129, -9, -81, 150, 66, 50, 13, -169, -64, 65, -15, 54, -155, 120, 257, -187, 442, 288, -335, 478, 40, -165, 104, 313, 12, -242, -121, 46, -13, 244, -432, 97, -125, 73, 129, -272, 114, 167, 102, -208, -73, 175, 7, 0, 90, -75, 16, 116, 153, 126, -65, 96, 281, 17, -249, -137, 60, -315, 104, -214, -132, 131, 35, 22, -217, -11, -248, 229, -36, -97, -114, -30, 305, 63, 36, 39, -53, -72, 195, -145, -44, -281, -221, 25, -171, -106, -174, -282, -111, -36, 372, 99, -198, 76, 48, -126, -51, 168, -71, 322, 76, -360, 67, -124, 281, -214, 123, 284, 229, -20, -15, -257, -33, 146, 198, -41, 32, 96, -115, -100, -1, -376, -142, -223, 81, 177, 158, 26, -37, -170, 295, 277, 145, 185, 42, -137, 237, -110, -62, 192, 5, -64, -102, -141, -145, -236, -249, -89, 300, 166, -31, -315, -52, 233, -25, 70, -327, 46, 57, -253, -56, 99, -233, 44, -194, -247, 156, 338, 137, 189, -3, -79, 192, -58, 0, -54, 134, 78, 28, -17, 119, 130, 281, 116, -262, -55, 421, -311, -19, 164, 242, -279, -121, -144, 63, 208, 20, -211, -46, 148, 158, -263, -59, -385, -238, -219, -138, 248, 223, -163, 38, -100, -75, 139, -41, -159, -79, 278, -93, -17, 220, 544, 328, -134, -285, -111, 231, 50, -246, 125, 56, 160, 49, 54, 48, 216, 166, -319, 140, -14, 161, -300, -27, 134, 176, 38, 246, -73, -73, -88, 32, 63, 124, -16, 432, -146, 44, 32, 119, -92, -151, -155, -20, 82, 35, -85, 186, 137, 151, -132, 36, 191, -64, -194, -162, 163, -453, 42, 36, -97, 84, 179, -337, 400, 20, -125, 158, -214, -124, 234, 254, -122, 119, -316, 383, 73, 30, 137, -131, 70, -260, -54, 7, -187, -183, -55, -192, -173, -13, 4, -127, -298, -94, 280, -232, 59, 38, -79, -46, -8, -204, -67, 146, -77, 60, 165, 9, -171, -276, 207, 166, -202, 128, -50, -91, 73, -48, 156, 29, -35, 143, 196, 34, -15, 154, -134, 206, -126, 82, -142, -103, 101, 195, 198, -161, -10, 373, 95, 56, 159, 263, -70, -121, 124, 49, -282, -122, 33, -18, -37, 164, -39, -269, 205, -335, -88, -96, -245, -225, -137, -31, 92, 312, -248, -98, 264, -101, -19, 46, 162, 274, -97, -6, -102, -101, 78, 21, -8, 124, 63, -214, -123, -209, 43, 54, 204, 21, 23, 178, 157, 45, 88, 326, 365, 8, 38, 182, -382, -98, -56, -154, 108, -229, 14, 172, -252, 187, 225, 156, -110, 15, -81, 97, 320, -70, 238, -57, 102, 4, -119, 39, -135, 183, 48, -25, 8, -155, 48, -35, -13, -107, 152, 328, -104, -254, -26, 57, 47, 69, 136, 184, -79, 170, 199, -167, 34, 3, 535]
s2= [185, -27, -106, 126, 36, 131, -136, -73, 114, -56, -97, 49, -115, 85, -103, 61, -115, -124, 88, -168, -291, -229, 322, 6, -97, -5, -133, 90, -150, 80, -98, 131, 34, 0, 112, -267, -323, 45, -113, 496, -85, 210, 160, 254, 278, 192, -177, -345, 74, 169, -223, -100, -43, -126, -229, -37, -77, -90, 101, -115, -75, 97, 138, -346, -72, -73, 46, 51, -3, 62, 247, -58, -86, 188, -39, 207, -5, 161, -123, 58, -404, -134, 59, -102, -28, -168, 81, 194, -93, -185, 53, -568, -100, -114, -101, -383, -49, -242, 71, 37, -208, 229, -48, -139, -127, -66, -295, -140, 12, -216, 12, -173, 15, -148, -168, 43, 10, -165, -125, -230, -252, 227, 271, 96, -134, 130, -49, 3, -135, -27, -291, 125, -157, 221, -350, -20, -199, 67, -74, 44, -113, -108, -281, 3, -156, -132, 364, 287, 183, 135, 80, 205, 370, 5, 51, -106, 301, 51, 190, -115, -94, 421, 59, 172, -322, -58, -225, 41, 75, 93, -130, 16, -80, 26, -28, 49, 75, -143, 41, 42, -317, -113, -282, -112, -145, -80, 169, 123, 8, -186, -32, 206, 4, 344, -90, 513, 146, -76, -167, -39, 0, 302, 123, 221, -191, -11, -73, 68, 202, -284, 287, 184, -248, 106, 160, -78, 113, 95, 28, 123, 58, 244, -306, -207, -59, 149, -181, -248, -89, -31, 403, -43, 102, -122, 176, 267, 92, -55, -62, 48, 275, 115, 22, 126, 97, 135, 46, -124, -136, -34, 99, 102, 47, -231, -143, 113, -145, -188, -207, -158, 78, 48, 83, -134, -324, 214, -97, -274, 105, 40, 90, -155, -8, 187, -49, 148, -108, -103, -81, -35, -33, -27, 229, 29, 201, -372, 36, -101, -222, 39, 72, -70, -64, 50, -190, 201, 72, 253, 58, -13, 222, -348, 47, 256, -3, -178, 51, 185, 52, 47, 95, -225, -89, 181, -38, -293, -227, 179, 45, -116, -90, -125, 32, -51, -152, -63, 10, 75, -46, 228, -148, 23, 2, 25, 25, 266, 1, 99, 31, -6, -139, 124, -110, 70, 54, 273, -330, 89, 112, 30, 50, 70, -162, -68, 44, 8, 380, 239, -225, 129, -215, 114, 68, -105, -6, 48, 224, 421, 63, -136, 203, 15, -32, -106, -58, -398, 90, 138, 169, 63, -99, -285, -192, -192, 302, 35, -187, -106, 60, 169, -20, -34, 252, -204, 144, -89, -50, 123, -194, 66, 59, -57, 98, -32, -24, -34, 165, 419, 206, -164, -330, 116, -63, 86, -15, -181, -102, 65, -7, -106, -27, 253, 39, 59, 9, -15, -135, 255, -158, 143, -240, 112, 122, 143, -159, 26, -144, 66, 67, -32, -19, 110, -199, -296, 211, -306, -232, -144, 74, 166, -167, 58, -54, 38, -96, -123, 265, -270, 41, -243, 48, -93, -174, -150, 66, 123, -101, 433, 97, 159, -29, 235, 102, 24, 6, 123, 293, 210, -41, 272, -210, 14, -247, -89, -80, 189, 183, -19, -87, 157, -443, 224, 18, -177, -87, 13, 152, 190, 468, -59, 227, 189, 281, -115, 122, -50, 191, 104, 22, 58, 184, -186]


In [615]:
len(s2)

512

In [616]:
r = b'\x00'*40

In [617]:
salt = b'\x00'*15

In [618]:
c = sk.hash_to_point(r,salt)

In [619]:
s1_ntt = ntt.ntt(s1)
print("s1")
print(s1[:10])
print(s1_ntt[:10])

s2_ntt = ntt.ntt(s2)
print("s2")
print(s2[:10])
print(s2_ntt[:10])

s1
[153, 89, 54, 219, 74, -306, 152, 77, -183, 76]
[709, 6605, 147, 9426, 5803, 5963, 8490, 5615, 6453, 9147]
s2
[185, -27, -106, 126, 36, 131, -136, -73, 114, -56]
[8842, 4060, 2868, 1589, 8168, 10636, 6497, 6176, 4584, 3348]
